Note: This jupyter notebook was adapted from my initial work done on a Google Collab space, therefore some implementations reflect that.

### Installations

The following are the necessary installations required to run this notebook.

Make sure you have the `requirements.txt` file!!!

Change the path accordingly to where your file is stored

#### Google Collab Steps

**Mounting Google Drive**

If on a Google Collab space, ensure to mount your Google drive to the Jupyter Notebook.

```python
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
```

**Install Requirements**

```python
%pip install -qU -r /content/requirements.txt
```

#### Launching Xterm

Xterm is a tool we will use to download open source Large Language Models that we can interact with.

Please follow the following instructions **carefully**

1. Run the code cell block

2. After running the cell block you should be directed to the Xterm terminal

3. Enter the following command in the console: `curl https://ollama.ai/install.sh | sh`

    * This is a command that will install ollama, a platform that allows you to serve open source large language models

    * This will take some time!

4. After succesfully downloading ollama we will begin to run the ollama server using the command `ollama serve &`

    * You should expect to see the following

        * `Couldn't find '/root ...'`

        * `Your new public key is:`

5. Back in the terminal in the space available type in the command `ollama pull llama3`

    * This should begin the process of downloading the open source large language model llama3

    * This will take a while!

6. Once the downloading is complete leave the terminal as is, you are now hosting the llama3 model through ollama and should be able to access it in the code steps

Launching Xterm Cell Block

```python
%xterm
```

#### Building Your Own Retrieval Augmented Generation Model

In the following steps you will build a Retrieval Augmented Generation Model using the open source Large Language Model llama3.

Please follow the steps carefully and answer all the provided questions IN YOUR OWN WORDS.

There is no need to use AI to answer these questions, the focus is not to be correct but to solidify your understanding of the concepts behind your implementation.


#### Setup

Please follow this [documentation](https://console.groq.com/docs/quickstart) to setup your API key.

Notes
* You will need to create a Groq account
* Once you have your Groq account you will need to create an API key that will allow you to access the LLMs stored on Groq
* DO NOT SHARE THIS API KEY WITH ANYONE!!!
* MAKE SURE YOU STORE THIS API KEY IN A SAFE PLACE!!!
* BE CAREFUL WHEN PROJECTS THAT INCLUDE YOUR API KEY TO GITHUB!!!
  * People search GitHub constantly for API KEYS that they can take advantage of

In [ ]:
import os
import getpass

#this will ask you to input your api key if it is not already stored in the environment
if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

We will use Groq to access the llama3 model stored on their servers for some aspects of this project in order to speed up the process, as using the llama3 instance served on Xterm would take too long.

In [ ]:
from langchain.chat_models import init_chat_model

#access the llama3 model stored on Groq
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

Answer the following questions in **your own words**

##### Question 1: What are embeddings and what function do they serve?

YOUR ANSWER HERE

##### Question 2: Why do we need to access the llama3 embeddings specifically?

YOUR ANSWER HERE

Next we will access the OllamaEmbeddings.

In [ ]:
from langchain_ollama import OllamaEmbeddings

#gather the ollama embeddings
embeddings = OllamaEmbeddings(model="llama3")

##### Question 3: In your own words what do you *think* is happening in the line of code below

YOUR ANSWER HERE

##### Question 4: Run a quick google search on vector stores and what purpose they serve in Large Langauge Models and re-answer question 3 below.

YOUR ANSWER HERE

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

#create a vector store
vector_store = InMemoryVectorStore(embeddings)

In this section we will load in the conversation file that you have edited
* Make sure this is a txt file!
* The path to the file that you upload might be different than the one stored below
  * Find the path and update accordingly if needed

In [ ]:
from langchain_unstructured import UnstructuredLoader

#gather file paths for augmented data
file_paths = [
    "/content/conversation.txt"
]
#build a loader object to load those files
loader = UnstructuredLoader(file_paths)
#load in the files into documents to be accessed
docs = loader.load()

Feeding entire files to train Large Language Models is extremely inefficient.

Train LLMs uses the process of chunking, where we feed in "chunks" of data at a time instead of all at once. Although our file is relatively small, we will use best practices and chunk our information that we want to feed into the LLM.

##### Question 5: Think of some the benefits of chunking and why we would need to do this to train Large Language Models

ANSWER HERE

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)

#use the text splitter to split our document into chunks
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Once our documents are split into chunks we store those chunks into the vector store that we created before.

##### Question 6: Why would we need to have id's associated with each chunk of our training data?

ANSWER HERE

In [ ]:
#add documents into our vector store
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

The following is a visualization of the prompts that are being sent under the hood to the large language models.

This is what the large language sees and uses to build a response.

When you type in your question to Chat-GPT, something like the object below is sent and used to build the response.

In [ ]:
from langchain import hub

#pull in standard rag prompt
prompt = hub.pull("rlm/rag-prompt")

#build an example prompt
example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
#display the prompt
print(example_messages[0].content)

The following class is used in the next methods to ensure that Data that is inputed to the RAG model is consistent.

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

#build state class that is used for retrieving and generating
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

##### Question 7: In your own words describe what you think the function *similarity_search* is doing. What role do embeddings have in this function?

ANSWER HERE

In [ ]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

##### Question 8: In your own words describe what the generate function is doing. What role does the output from the previous function *retrieve* have in this?

ANSWER HERE

In [ ]:
def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

Below we build a LangChain Graph. You can think of this Graph as a way to create a workflow with multiple sequences. The image below shows you the steps within the created workflow. LangGraphs can be used with Agents but in this case we will use them with a RAG system.